In [3]:
# !pip install transformers
# !pip install numpy==1.26.0
# !pip install tensorflow[and-cuda]
# !pip install torch torchvision torchaudio
# !pip install --upgrade ipywidgets
# !pip install tf-keras
# !pip install pandas
# !pip install scikit-learn
!pip install torchinfo

In [4]:
!nvidia-smi

Sat Mar  8 23:20:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!git clone https://github.com/avyas21/interpretablellm.git

fatal: destination path 'interpretablellm' already exists and is not an empty directory.


In [6]:
%cd interpretablellm
!ls

/content/interpretablellm
baseline.ipynb	baseline_prompt.txt  data  README.md  setup_dataset.ipynb


## Data Preprocessing

In [7]:
import pandas as pd

In [8]:
train_data = pd.read_csv("data/train_data.csv")
test_data = pd.read_csv("data/test_data.csv")

In [9]:
POSITIVE_WORDS = ["positive", "great", "good", "happy", "amazing", "fantastic", "yes"]
NEGATIVE_WORDS = ["negative", "bad", "sad", "terrible", "horrible", "no", "critical"]

In [10]:
def convert_lbl_to_int(label):
    if label.lower() in POSITIVE_WORDS:
        return 1
    if label.lower() in NEGATIVE_WORDS:
        return 0
    return -1

## Baseline Model

In [11]:
import numpy as np
print(np.__version__)
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, BertConfig
from transformers import pipeline
from sklearn.metrics import f1_score
from torchinfo import summary
import random

1.26.4


In [12]:
def get_prompt(review):
  with open("baseline_prompt.txt", "r") as file:
    prompt = file.read()
  prompt = prompt.replace("<REVIEW>", review)
  return prompt

print(get_prompt("Test Prompt"))

Given a book review, classify it as expressing a positive or negative sentiment.
Review: Test Prompt
This review is either positive or negative sentiment. If one had to chosen, the sentiment in the review is [MASK].



In [13]:
# Load the BERT model and tokenizer
baseline_model = BertModel.from_pretrained("bert-base-uncased")
baseline_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
baseline_unmasker = pipeline('fill-mask', model='bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/do

In [14]:
def predict(review, unmasker):
  prompt = get_prompt(review)
  predictions = unmasker(prompt)
  valid_predictions = POSITIVE_WORDS + NEGATIVE_WORDS

  for prediction in predictions:
    if(prediction['token_str'] in valid_predictions):
      return prediction['token_str']

  for prediction in predictions:
    print(prediction['token_str'])

  sentiment = ["positive", "negative"]
  #If not found, lets predict random
  return "NOT FOUND" #random.choice(sentiment)

In [15]:
def predict_baseline(df, model, tokenizer, unmasker):
    predictions = []
    labels = []
    for idx, row in df.iterrows():
        input_text = row['Review']
        prediction = predict(input_text, unmasker)
        predictions.append(convert_lbl_to_int(prediction))
        labels.append(convert_lbl_to_int(row['Sentiment']))
    return predictions, labels



In [16]:
def score_baseline(baseline_model, df, baseline_tokenizer, baseline_unmasker):
    predictions, labels = predict_baseline(df, baseline_model, baseline_tokenizer, baseline_unmasker)
    values, counts = np.unique(np.array(predictions), return_counts=True)

    for v, c in zip(values, counts):
        print(f"Value: {v}, Count: {c}")

    return f1_score(labels, predictions, average='micro')


In [14]:
train_baseline_f1 = score_baseline(baseline_model, train_data, baseline_tokenizer, baseline_unmasker)
print(train_baseline_f1)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


KeyboardInterrupt: 

In [ ]:
test_baseline_f1 = score_baseline(baseline_model, test_data, baseline_tokenizer, baseline_unmasker)
print(test_baseline_f1)

## Probe Models

In [21]:
import torch.nn as nn
import torch
from transformers import AutoTokenizer, BertModel
import pandas as pd

class CustomBERTModel(nn.Module):
    def __init__(self, n, randomized=False, linear_1 = None, linear_2 = None):
        super(CustomBERTModel, self).__init__()
        if randomized:
          config = BertConfig()
          self.bert = BertModel(config).to(torch.device("cuda"))
        else:
          self.bert=  BertModel.from_pretrained("bert-base-uncased")
        self.bert.encoder.layer = nn.ModuleList(self.bert.encoder.layer[:n])

        # Reinitialize weights for each layer in the encoder
        for layer in self.bert.encoder.layer:
            for module in layer.modules():
              for mod in module.modules():
                  if isinstance(mod, (nn.Linear, nn.Conv2d)):
                      # Reinitialize weights for linear and convolution layers
                      # print("Before Randomization:", mod.weight)
                      nn.init.xavier_uniform_(mod.weight)
                      # print("After Randomization:", mod.weight)
                      if mod.bias is not None:
                          nn.init.zeros_(mod.bias)
                  elif isinstance(mod, nn.LayerNorm):
                      # Reinitialize LayerNorm layers
                      nn.init.ones_(mod.weight)
                      nn.init.zeros_(mod.bias)
                  # print("Mod")
                  # print(mod)
              #print(module)
            #print(layer)

        for param in self.bert.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(0.5)

        ### New layers:

        if linear_1 is not None:
          self.linear_1 = linear_1
        else:
          self.linear_1 = nn.Linear(768, 256)

        if linear_2 is not None:
          self.linear_2 = linear_2
        else:
          self.linear_2 = nn.Linear(256, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, ids, mask):
        output = self.bert(input_ids=ids, attention_mask=mask, output_hidden_states=True)

        linear1_output = self.linear_1(output.last_hidden_state[:,0,:])
        linear2_output = self.linear_2(self.dropout(linear1_output))
        sigmoid_output = self.sigmoid(linear2_output)
        return sigmoid_output


In [22]:
def get_custom_bert_model(num_bert_layers, toggle):
  gpu_available = torch.cuda.is_available()
  model = CustomBERTModel(num_bert_layers, toggle)
  if gpu_available:
    return model.to(torch.device("cuda"))
  return model

In [23]:
baseline_model = get_custom_bert_model(2, True)

In [24]:
print(baseline_model.linear_1)

Linear(in_features=768, out_features=256, bias=True)


In [122]:
def get_inputs_labels(df):
    inputs = []
    labels = []
    for idx, row in train_data.iterrows():
        input_text = row['Review']
        inputs.append(input_text)
        labels.append(1 if row['Sentiment'] == 'positive' else 0)
    return inputs, labels


In [123]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }

In [132]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
inputs, labels = get_inputs_labels(train_data)
dataset = TextDataset(inputs, labels, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [125]:
def predict(model, df):
    model.eval()
    predictions = []
    labels = []
    gpu_available = torch.cuda.is_available()
    with torch.no_grad():
        for idx, row in df.iterrows():
            input_text = row['Review']
            encoding = tokenizer(input_text, add_special_tokens=True, max_length = 512, padding='max_length', truncation=True, return_tensors='pt')
            if gpu_available:
              input_ids = encoding['input_ids'].cuda()
              attention_mask = encoding['attention_mask'].cuda()
              prediction = model(encoding['input_ids'].cuda(), encoding['attention_mask'].cuda())
            else:
              input_ids = encoding['input_ids']
              attention_mask = encoding['attention_mask']
              prediction = model(encoding['input_ids'], encoding['attention_mask'])
            predictions.append(1 if prediction > 0.5 else 0)
            labels.append(convert_lbl_to_int(row['Sentiment']))

    return predictions, labels

In [126]:
def score_model(model, train_df, test_df):
    train_f1 = None
    test_f1 = None

    if train_df is not None:
        train_predictions, train_labels = predict(model, train_df)
        train_f1 = f1_score(train_labels, train_predictions, average='micro')

    if test_df is not None:
        test_predictions, test_labels = predict(model, test_df)
        test_f1 = f1_score(test_labels, test_predictions, average='micro')

    return train_f1, test_f1

# print(score_custom_model(model, train_data, test_data))

In [127]:
def get_loss(model, df):
    inputs, labels = get_inputs_labels(df)

In [128]:
print(torch.cuda.is_available())

True


In [133]:
def train_model(model, epochs, dataloader, train_df, learning_rate = 1e-3, calc_train_f1 = True):
    criterion = nn.BCELoss() ## If required define your own criterion
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = learning_rate)
    gpu_available = torch.cuda.is_available()

    for epoch in range(epochs):
        i = 0
        for batch in dataloader:
            targets = np.array(batch['label'])
            targets = torch.tensor(np.expand_dims(targets,axis=1)).float()
            optimizer.zero_grad()
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']

            if gpu_available:
              targets = targets.cuda()
              input_ids = input_ids.cuda()
              attention_mask = attention_mask.cuda()

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, targets)
            loss.backward()
            # if i % 20 == 0:
            #   print(loss)
            optimizer.step()
            i += 1

        if calc_train_f1:
          train_f1, _ = score_model(model, train_df, None)
          print("Epoch: " + str(epoch) + " F1: " + str(train_f1) + " LOSS: " + str(loss))

In [116]:
train_model(baseline_model, 10, dataloader, train_data)

tensor(0.7615, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.9821, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6292, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7170, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6782, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7056, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6367, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7618, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7097, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6708, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6675, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.5491, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6323, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 0 F1: 0.64725 LOSS: tensor(0.7356, device='cuda:0', grad_fn=<BinaryCro

In [117]:
_, test_f1 = score_model(baseline_model, None, test_data)
print("TEST F1: " + str(test_f1))

TEST F1: 0.6005


In [130]:
def score_all_randomized_models(dataloader, train_df, test_df, epochs, max_n, learning_rate = 1e-3):
    model_scores = []
    for n in range(1,max_n + 1):
        print("N: " + str(n))
        model = get_custom_bert_model(n, True)
        train_model(model, epochs, dataloader, train_df, learning_rate)
        _, test_f1 = score_model(model, None, test_df)
        print("TEST F1: " + str(test_f1))
        model_scores.append([n, test_f1])

    return model_scores

In [ ]:
model_scores = score_all_randomized_models(dataloader, train_data, test_data, 10, 12)

N: 1
Epoch: 0 F1: 0.618875 LOSS: tensor(0.6379, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-134-70b63bbe3862>", line 1, in <cell line: 0>
    model_scores = score_all_randomized_models(dataloader, train_data, test_data, 10, 12)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-130-bc5c5454ae21>", line 6, in score_all_randomized_models
    train_model(model, epochs, dataloader, train_df, learning_rate)
  File "<ipython-input-133-fd5dc9de79d6>", line 29, in train_model
    train_f1, _ = score_model(model, train_df, None)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-126-c3b53445eb9a>", line 6, in score_model
    train_predictions, train_labels = predict(model, train_df)
                                      ^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-125-392c8d43dee1>"

In [26]:
def score_all_probe_models(dataloader, train_df, test_df, epochs, max_n, learning_rate = 1e-3):
    model_scores = []
    for n in range(1,max_n + 1):
        print("N: " + str(n))
        model = get_custom_bert_model(n)
        train_model(model, epochs, dataloader, train_df, learning_rate)
        _, test_f1 = score_model(model, None, test_df)
        print("TEST F1: " + str(test_f1))
        model_scores.append([n, test_f1])

    return model_scores

## Scalar Mixing Weights

In [31]:
import torch.nn as nn
import torch
from transformers import AutoTokenizer, BertModel
import pandas as pd

class ScalarMixingWeightModel(nn.Module):
    def __init__(self, n, i):
        super(ScalarMixingWeightModel, self).__init__()

        self.bert= BertModel.from_pretrained("bert-base-uncased")
        self.bert.encoder.layer = nn.ModuleList(self.bert.encoder.layer[:n])
        self.n = n
        self.layer_weights = nn.Parameter(torch.ones(self.n))
        self.softmax = nn.Softmax(dim=0)
        self.i = i

        self.gamma = nn.Parameter(torch.ones(1))
        self.dropout = nn.Dropout(0.5)

        for param in self.bert.parameters():
            param.requires_grad = False

        ### New layers:
        if self.i == 1:
          self.linear1 = nn.Linear(768, 1)
        elif self.i == 2:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 1)
        elif self.i == 3:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 192)
          self.linear3 = nn.Linear(192, 1)
        elif self.i == 4:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 192)
          self.linear3 = nn.Linear(192, 96)
          self.linear4 = nn.Linear(96, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, ids, mask):
        outputs = self.bert(input_ids=ids, attention_mask=mask, output_hidden_states=True)

        hidden_states = outputs.hidden_states[1:1 + self.n]

        normalized_weights = self.softmax(self.layer_weights)
        scalar_mixing_weight = self.gamma * sum(normalized_weights[i] * hidden_states[i] for i in range(self.n))

        if self.i == 1:
          linear1_output = self.dropout(self.linear1(scalar_mixing_weight[:, 0, :]))
          sigmoid_output = self.sigmoid(linear1_output)
        elif self.i == 2:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          sigmoid_output = self.sigmoid(linear2_output)
        elif self.i == 3:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          linear3_output = self.linear3(self.dropout(linear2_output))
          sigmoid_output = self.sigmoid(linear3_output)
        elif self.i == 4:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          linear3_output = self.linear3(self.dropout(linear2_output))
          linear4_output = self.linear4(self.dropout(linear3_output))
          sigmoid_output = self.sigmoid(linear4_output)

        return sigmoid_output


In [32]:
def get_scalar_mixing_model(num_bert_layers, i):
  gpu_available = torch.cuda.is_available()
  model = ScalarMixingWeightModel(num_bert_layers, i)

  if gpu_available:
    return model.to(torch.device("cuda"))

  return model

In [33]:
weights = {}
for i in range(11,13):
  test_scalar = get_scalar_mixing_model(i,1)
  train_model(test_scalar, 10, dataloader, train_data, learning_rate = 0.1)
  print(i, ":", test_scalar.layer_weights)
  weights[i] = test_scalar.layer_weights

tensor(0.6915, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(19.0966, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(37.8466, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(12.8466, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(22.2649, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(34.7649, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(15.9716, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(12.9332, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(19.1399, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(28.4716, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(19.0749, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(19.1182, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(19.0749, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 0 F1: 0.5 LOSS: tensor(37.7166, device='cuda:0', grad_fn=<

KeyboardInterrupt: 

In [ ]:
lrs = [0.1, 0.01, 0.001, 0.0001]
for lr in lrs:
  for i in range(1,5):
    test_scalar = get_scalar_mixing_model(2, i)
    train_model(test_scalar, 10, dataloader, train_data, learning_rate = lr)
    print("Learning rate:", lr, "Number of layers", i)

Epoch: 0 F1: 0.5 LOSS: tensor(0.6902, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 1 F1: 0.579 LOSS: tensor(0.6880, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 2 F1: 0.747375 LOSS: tensor(0.6665, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 3 F1: 0.76175 LOSS: tensor(0.4873, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 4 F1: 0.754875 LOSS: tensor(0.4700, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 5 F1: 0.771375 LOSS: tensor(0.3286, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 6 F1: 0.817375 LOSS: tensor(0.4258, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 7 F1: 0.808375 LOSS: tensor(0.3882, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 8 F1: 0.7935 LOSS: tensor(0.6432, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 9 F1: 0.827375 LOSS: tensor(0.3705, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Learning rate: 0.1 Nu

## Analysis

In [ ]:
summary(baseline_model)

Layer (type:depth-idx)                                  Param #
BertModel                                               --
├─BertEmbeddings: 1-1                                   --
│    └─Embedding: 2-1                                   (23,440,896)
│    └─Embedding: 2-2                                   (393,216)
│    └─Embedding: 2-3                                   (1,536)
│    └─LayerNorm: 2-4                                   (1,536)
│    └─Dropout: 2-5                                     --
├─BertEncoder: 1-2                                      --
│    └─ModuleList: 2-6                                  --
│    │    └─BertLayer: 3-1                              (7,087,872)
│    │    └─BertLayer: 3-2                              (7,087,872)
│    │    └─BertLayer: 3-3                              (7,087,872)
│    │    └─BertLayer: 3-4                              (7,087,872)
│    │    └─BertLayer: 3-5                              (7,087,872)
│    │    └─BertLayer: 3-6            

In [ ]:
summary( get_custom_bert_model(10))

Layer (type:depth-idx)                             Param #
CustomBERTModel                                    --
├─BertEmbeddings: 1-1                              --
│    └─Embedding: 2-1                              (23,440,896)
│    └─Embedding: 2-2                              (393,216)
│    └─Embedding: 2-3                              (1,536)
│    └─LayerNorm: 2-4                              (1,536)
│    └─Dropout: 2-5                                --
├─ModuleList: 1-2                                  --
│    └─BertLayer: 2-6                              --
│    │    └─BertAttention: 3-1                     (2,363,904)
│    │    └─BertIntermediate: 3-2                  (2,362,368)
│    │    └─BertOutput: 3-3                        (2,361,600)
│    └─BertLayer: 2-7                              --
│    │    └─BertAttention: 3-4                     (2,363,904)
│    │    └─BertIntermediate: 3-5                  (2,362,368)
│    │    └─BertOutput: 3-6                        (2,361,6